**DEVOIR 1 - ÉTUDE EMPIRIQUE DU PROBLÈME DU SAC À DOS**

**1. Information sur le groupe et le rapport**

Numéro du Groupe: 1\\
Nom des membres du groupe: Yasmine Zoubdi et Tyler\\
Numéros d'étudiants des membres du groupe : 300170464 et 300169866\\

**2. Problème du Sac à dos (Knapsack)**

Le problème que nous allons traiter aujourd'hui est le problème du sac à dos, qui consiste à choisir un sous-ensemble d'objets ayant des poids et des valeurs différents, de manière à maximiser la valeur totale des objets tout en respectant la contrainte de poids d'un sac à dos ayant une capacité limitée.

**3. Ensemble de données**

Les données que nous allons utiliser proviennent du site [Kaggle](https://www.kaggle.com/datasets/warcoder/knapsack-problem/data). Cet ensemble de données se compose de 5 colonnes et 10 000 lignes, où chaque ligne représente un problème de sac à dos distinct avec sa solution optimale.

**Importation des bibliothèques importantes**

In [ ]:
import pandas as pd
import itertools
import numpy as np

**Lire l'ensemble de données**

In [ ]:
url = "https://raw.githubusercontent.com/Yasmine08POG/CSI4506_A1/main/knapsack_5_items.csv"

dataset = pd.read_csv(url)

Regardons quelles sont les colonnes du dataset :

In [ ]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

Comme nous nous y attendions, nous avons des colonnes pour les poids, les coûts, la capacité, les meilleurs choix et le meilleur prix pour toutes les instances.

Regardons les 10 premières entrées (rangées):

In [ ]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Étape de pré-traitenemtn**

Règle générale, la première étape de tout projet impliquant la lecture et la manipulation de données est le prétraitement et le nettoyage des données.

Dans notre ensemble de données, nous nous attendons à ce que les entrées des colonnes « Weights », « Prices » et « Best Picks » se présentent sous la forme de tableaux de nombres réels ou d'entiers, comme ceci : [45, 40, 42, 38, 10]

Cependant, lorsque vous lisez les entrées à l'aide de pandas, elles se présentent sous la forme de chaînes (String): "[45 40 42 38 10]"

Nous devons donc convertir ces chaînes en « tableaux de réels ou d’entiers ». Vous pouvez utiliser la fonction fournie ci-dessous à cet effet :


In [ ]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list


De plus, il est possible que certaines lignes de l'ensemble de données contiennent des valeurs vides dans des colonnes spécifiques. Nous voulons également éliminer ces lignes car elles ne fournissent aucune information utile. Nous utilisons la fonction dropna() pour ce faire :

In [ ]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

<ipython-input-5-833fb421dc49>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
<ipython-input-5-833fb421dc49>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
<ipython-input-5-833fb421dc49>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document


Il est maintenant temps de mettre en œuvre les algorithmes de recherche. Pour chaque algorithme, un modèle vous est fourni. Vous pouvez modifier ce modèle si vous voulez, mais regardez d'abord tous les paramètres utilisés car ils sont tous importants. Vous pouvez également définir autant de fonctions auxiliaires que vous le souhaitez.

**4. Generate and Test**

L'algorithme de génération et de test génère toutes les solutions possibles puis les évalue pour trouver la meilleure.

In [ ]:
def gen_and_test(data):
    best_solution = None
    best_solution_price = 0

    # Generate all possible combinations of items
    AllCombinations = list(itertools.product(range(2), repeat=len(data.Weights)))
    # print(AllCombinations)
    #Calculate Total Weight and Total value
    for combination in AllCombinations :
        total_weight = sum([a * b for a, b in zip(data.Weights , combination)])
        total_price = sum([a * b for a, b in zip(data.Prices , combination)])

        # Test if combination fits within capacity
        if total_weight <= data.Capacity:
          if total_price > best_solution_price:
              best_solution = combination
              best_solution_price = total_price
    return best_solution_price, best_solution



In [ ]:
solutions = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)


In [ ]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 1.0


**Votre analyse:**

La précision obtenue est de 1, ce qui signifie que les résultats produits par le code correspondent toujours à la solution optimale attendue. L'exécution du code ne dépasse pas 9 secondes, ce qui est très rapide pour un algorithme de génération et de test.


------------------------------------------------------------------------------------------------

**5. Recherche gloutonne (greedy search)**

Greedy algorithms work by making locally optimal decisions hoping for a globally optimal solution. It's not always obvious what parameter to use to find the best choice every time, but for this one we decided to use the price/weight as the optimal decision.

In [ ]:
def greedy(data):

  best_solution_price = 0

  solList =[0] * len(data.Weights)
  #create an array containg all the price/weight ratios
  ratio = []
  for i in range(len(data.Prices)):
    ratio.append(data.Prices[i]/data.Weights[i])
  best = 0
  index = []
  #create an array containing all the index number of the largest ratio first and the smallest one last
  for i in range(len(ratio)):
        index.append(ratio.index(max(ratio)))
        ratio[ratio.index(max(ratio))] = 0

  #since we now know the index of the best price/weight items, we add them in descending order, always checking that we don't go over capacity
  for i in index:
    if(data.Capacity - data.Weights[i] > 0):
      best_solution_price += data.Prices[i]
      data.Capacity -= data.Weights[i]
      solList[i] =1

  best_solution = solList


  return best_solution_price, best_solution

In [ ]:
solutions_greedy = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)


In [ ]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

Greedy Accuracy is 0.7808482639943314


**Votre analyse:**

For the greedy algorithm price/weight was used to calculate the bet choice at every step. The greedy algorithm got a lower accuracy score than generate and test, but it did run more than twice as quickly and it scales better as the problems increase in size.

------------------------------------------------------------------------------------------------

**6. Recuit Simulé (Simulated Annealing)**

Simulated annealing works by making random changes to our data and hopping that the locally bad decisions help us reach an overall better solution further down the line. We decided to use the greedy algorithm as a base before randomly changing bits and using the temperature and the difference in prices to see if the changes are worth exploring.


In [ ]:
import random
import math


def simulated_annealing(data, N, initial_temperature, cooling_rate):
  #We use the greedy algorith that was made in problem 5 to generate our base case
  currPrice, currSolution = greedy(data)

  best_solution = currSolution.copy()
  best_solution_price = currPrice
  temperature = initial_temperature
  currWeight = 0
  maxWeight = data.Capacity
  neighbourPrice = 0
  for a in range(N):
    #This rolls back unwanted changes in case the code decided it wanted to revert the change back in line 45
    neighbourSolution = currSolution.copy()
    currWeight = 0
    neighbourPrice = 0

    #generate a random index number then flip that bit
    rand = random.randint(0, len(currSolution)-1)
    neighbourSolution[rand] = 1 - neighbourSolution[rand]

    #calculate the new weight
    for i in range(len(neighbourSolution)):
      if(neighbourSolution[i]==1):
        currWeight += data.Weights[i]

    #if the weight is pass the capacity, set the score to zero, if not calculate the new score
    if(currWeight > maxWeight):
      neighbourPrice = 0
    else:
      for i in range(len(currSolution)):
        if(neighbourSolution[i] ==1):
          neighbourPrice += data.Prices[i]

    #If by fliping that bit we get a better price, then set the new best price to that, and carry the changes to the next iteration
    if neighbourPrice > best_solution_price:
      best_solution_price = neighbourPrice
      best_solution = neighbourSolution.copy()
      currPrice = neighbourPrice
      currSolution = neighbourSolution.copy()
    else:
      #else, get a % chance based off the current heat and price fluctuation to keep the changes to see if they help further down
      probability = math.exp((neighbourPrice - currPrice)/temperature)
      if(random.random() < probability):
        currPrice = neighbourPrice
        currSolution = neighbourSolution.copy()

    temperature *= cooling_rate



  return best_solution_price, best_solution


In [ ]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)


In [ ]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

Simulated Annealing Accuracy is 0.7818605121975909


In [ ]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 20, initial_temperature=1, cooling_rate=0.99)
    solutions_sa.append(1 if target == solution else 0)

In [ ]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

In [ ]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.8)
    solutions_sa.append(1 if target == solution else 0)

In [ ]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

**Votre analyse:**

*The* simulated annealing is slightly better than the greedy algorithm, which is used as a base for this one, but the accuracy is only about 0.02 better. As N and the cooling rate get bigger, the accuracy does seem to slightly improve.

------------------------------------------------------------------------------------------------

**7. Algorithme génétique**

L'algorithme génétique est une méthode d'optimisation inspirée par la sélection naturelle. L'application de cet algorithme au problème du sac à dos conciste a crée une population de solutions potentielles aléatoire, les évalue en fonction de leur aptitude à respecter les contraintes de poids et maximiser le prix totale, puis les combine et modifie pour générer de nouvelles solutions. Ce processus itératif se poursuit jusqu'à ce le nombre maximum de générations soit atteint.

In [ ]:
import random

def calculate_fitness(ind, prices, weights, capacity): #determines how fit an individual is (returns a fitness score)
  total_price = sum([a * b for a, b in zip(prices , ind)])
  total_weight = sum([a * b for a, b in zip(weights , ind)])

  if total_weight > capacity :
    fitness = 0
  else:
    fitness = total_price
  return fitness

def crossover(parent1, parent2, cross_rate):
  if random.random() < cross_rate: #check if crossover will happen depending on crossover rate
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point]+ parent2[crossover_point:]
    child2 = parent2[:crossover_point]+ parent1[crossover_point:]

  else:
    child1 = parent1
    child2 = parent2
  return child1, child2

def mutation(child, mut_rate):
  if random.random() < mut_rate: #check if mutation will happen depending on mutation rate
    #Generate random mutation point
    mutation_point = random.randint(1, len(child) - 1)

    #Flip values at the mutation point
    child[mutation_point] = 1 - child[mutation_point]
  return child

def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):

  #Generate random population of size population_size
  population =  [[random.randint(0, 1) for _ in range(len(data.Weights))]for _ in range(population_size)]

  for i in range(num_generations):
    ## Tournament 1
    #A total of tournament_size individuals are randomly selected
    tournament =  random.sample(population, tournament_size)

    winner1 = tournament[0]
    for ind in tournament :
      if calculate_fitness(ind,data.Prices,data.Weights,data.Capacity) >= calculate_fitness(winner1,data.Prices,data.Weights,data.Capacity):
        winner1 = ind

    ## Tournament 2
    #A total of tournament_size individuals are randomly selected
    tournament =  random.sample(population, tournament_size)

    winner2 = tournament[0]
    for ind in tournament :
      if calculate_fitness(ind,data.Prices,data.Weights,data.Capacity) >= calculate_fitness(winner2,data.Prices,data.Weights,data.Capacity):
        winner2 = ind

    #Create new children and add it to the population
    child1 , child2 = crossover(winner1,winner2,cross_rate)
    child1 = mutation(child1,mut_rate)
    child2 = mutation(child2,mut_rate)
    population.append(child1)
    population.append(child2)



  best_solution = max(population ,key=lambda x: calculate_fitness(x,data.Prices,data.Weights,data.Capacity))
  best_solution_price = sum([a * b for a, b in zip(data.Prices , best_solution)])


  return best_solution_price, best_solution

In [ ]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.1, cross_rate = 0.7, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)


In [ ]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

Genetic Algorithm Accuracy is 0.9390626581637818


In [ ]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.3, cross_rate = 0.9, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)

In [ ]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

Genetic Algorithm Accuracy is 0.9686203056989574


In [ ]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.05, cross_rate = 0.5, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)

In [ ]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

Genetic Algorithm Accuracy is 0.9142625771839255


**Votre analyse:**

* La précision obtenue pour l'algorithme génétique est de 94 %. Cela indique que les solutions obtenues sont correctes dans 94 % des cas. Cependant, on remarque que le temps d'exécution s'élève à 4 minutes, ce qui est considérablement long pour un problème simple de Sac à dos.


*   Lorsque l'on modifie les taux de croisement et de mutation, on remarque que la précision est plus élevée pour des taux plus grands que ceux par défaut, tandis que la précision est plus basse pour des taux plus petits que ceux par défaut.




----------------

**8. Étude comparative**

| Algorithme           | Paramètres | Temps d'execution | Précision  |
|----------------------|------------|-------------------|------------|
| Recherche gloutonne  | __         |       3s            |   0.7808482639943314         |
| Recruit Simulé       |N = 10, cooling_rate=0.95            |     6s              |   0.7822654114788946         |
| Recruit Simulé       |N = 20, cooling_rate=0.99            |      9s             | 0.7825690859398725           |
| Recruit Simulé       |N = 10, cooling_rate=0.80         |   6s                |   0.7819617370179168         |   
| Algorithme génétique | mut_rate = 0.1, cross_rate = 0.7|4m                   |  0.9388602085231299          |
| Algorithme génétique | mut_rate = 0.3, cross_rate = 0.9| 4m                  |  0.9696325539022168          |
| Algorithme génétique | mut_rate = 0.05, cross_rate = 0.5|   4m                |       0.9120356311367548     |
                

* Parmi tous les algorithmes testés, l'algorithme glouton était le plus rapide, mais aussi le moins précis, bien que la différence ne soit pas très importante.

* L'algorithme du recuit simulé était un peu plus précis que l'algorithme glouton mais avec un temps d'exécution double.

* l'algorithme génétique étaient de loin le plus lents parmi tous les algorithmes, mais aussi le plus précis.

* En ce qui concerne les paramètres :
  * Pour l'algorithme du recuit simulé, on remarque que l'exactitude s'améliorait à mesure que le nombre de cycles et le taux de refroidissement augmentaient. Cependant, le nombre de cycles augmentait également considérablement le temps d'exécution.
  * Pour l'algorithme génétique,  on remarque que l'exactitude s'améliorait à mesure que les taux augmentaient. Cependant, les paramètres ne semblent pas avoir d'effet sur le temps d'exécution.



--------------------------------------------------------------------------


**9. Conclusion**

Ce devoir nous a permis de développer et de comparer quatre algorithmes pour résoudre le problème du sac à dos. Nous avons observé que le temps d'exécution et la précision varient en fonction de l'algorithme choisi et des paramètres utilisés. Une conclusion intéressante est que, contrairement à la réputation de l'algorithme de génération et de test en tant qu'algorithme lent, il s'est avéré être le plus performant en termes de temps d'exécution et de précision ensembles.

Cela prouve que la permance d'un algorithme depends du problem à resoudre. Il est donc important de prendre en compte les caractéristiques des données ainsi qu'effectuer plusieurs expérimentations et tests pour déterminer quelle approche fonctionne le mieux dans un contexte donné.

--------------------------------------------------------------------------


**10. Références**

[1] https://www.geeksforgeeks.org/simulated-annealing/

[2] https://www.geeksforgeeks.org/genetic-algorithms/

[3] https://docs.python.org/3/library/itertools.html



**Hint:**

Pour partager un lien vers votre notebook Colab, cliquez sur "share" en haut à droite. Ensuite, sous *General access* , remplacez *Restricted* par « Anyone with the link ».